In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import os
# from dotenv import load_dotenv

# load_dotenv(os.path.expanduser("~/.nmdc_mongo.env"))

In [ ]:
from dotenv import load_dotenv

load_dotenv("../../.env.localhost")

TODO

- data object `data_object_type` field with [enum](https://docs.google.com/document/d/1uq6J__NbCUezsD16VTVJa-3_HJn5DacASBmFw3xQ3Bc/edit) codes

In [ ]:
from nmdc_runtime.pipelines.core import preset_normal_env
from nmdc_runtime.resources.mongo import get_mongo


db = get_mongo(preset_normal_env.run_config).db

In [ ]:
db.list_collection_names()

In [ ]:
docs = []

In [ ]:
functions = set(db["raw.functional_annotation_set"].distinct("has_function"))

In [ ]:
len(functions)

In [ ]:
import json

with open('functional_annotation_set.distinct.has_function.json', 'w') as f:
    json.dump(sorted(functions), f, indent=2)

In [ ]:
!head ~/Desktop/functional_annotation_set.distinct.has_function.json

In [ ]:
doc = db["raw.functional_annotation_set"].find_one()
doc["has_function"]

In [ ]:
len([f for f in functions if ":" not in f])

In [ ]:
22635 / 60584

In [ ]:
(db["raw.functional_annotation_set"]
 .count_documents({"has_function": {"$regex": "^KEGG\.ORTHOLOGY"}}))

In [ ]:
db["raw.functional_annotation_set"].estimated_document_count()

In [ ]:
docs.extend(list(db["raw.functional_annotation_set"]
                 .find({"has_function": {"$regex": "^KEGG"}})
                 .limit(3)
                ))

In [ ]:
docs

In [ ]:
# from pydantic import BaseModel

# class PatchDoc(BaseModel):
#     # id of doc in original collection, or new doc id
#     id: str
#     # JSON patch (JSON array of JSON objects)
#     patch: list
#     # array of mongodb filters, all of which must return non-zero count_documents
#     allow: list = []
#     # array of mongodb filters, none of which must return non-zero count_documents        
#     deny: list = []

In [ ]:
import json
import re
from toolz import assoc_in, dissoc
from zipfile import ZipFile

from mongospawn.schema import collschemas_for

from nmdc_mongo import (
    add_to_db,
#     correct_metaP_doc,
#     dbschema,
#     fetch_and_validate_json,
#     fetch_conform_and_persist_from_manifest,
#     fetch_json,
#     get_db,
#     reset_database,
#     snake_case_set_name
)

In [ ]:
###########################
# Adjustments for GSP below
###########################

defined_object_names = set(dbschema["definitions"])

set_for_object_name = {
    spec["items"]["$ref"].split("#/definitions/")[-1]: set_name
    for set_name, spec in dbschema["properties"].items()
}

existing_set_names = set(dbschema["properties"])

for object_without_set in (defined_object_names - set(set_for_object_name.keys())):
    proposed_set_name = snake_case_set_name(object_without_set)
    if proposed_set_name not in existing_set_names:
        dbschema["properties"][proposed_set_name] = {
            "description": (f"This property links a database object to the set of"
                            f" {object_without_set} objects within it."),
            "items": {"$ref": f"#/definitions/{object_without_set}"},
            "type": "array",
        }
        
dbschema = assoc_in(dbschema, ["definitions", "ControlledTermValue", "properties", "term", "type"], "string")
del dbschema["definitions"]["ControlledTermValue"]["properties"]["term"]["$ref"]
dbschema = assoc_in(dbschema, ["definitions", "ControlledTermValue", "properties", "term", "type"], "string")


# 'k' not capitalized upstream perhaps. should conform!
#dbschema = assoc_in(dbschema, ["definitions", "MetagenomeAssembly", "properties", "scaf_l_gt50k", "type"], "number")

In [ ]:
collschemas = collschemas_for(dbschema)

# Reconstruct
set_for_object_name = {
    spec["items"]["$ref"].split("#/definitions/")[-1]: set_name
    for set_name, spec in dbschema["properties"].items()
}

In [ ]:
import json
import re
from toolz import assoc_in, dissoc
from zipfile import ZipFile

with ZipFile('/Users/dwinston/Desktop/nmdc_database.json.zip') as myzip:
    # may be e.g. 'metadata-translation/src/bin/output/nmdc_database.json' rather than 'nmdc_database.json'
    name = next(n for n in myzip.namelist() if n.endswith("nmdc_database.json"))
    with myzip.open(name) as f:
        nmdc_database = json.load(f)

In [ ]:
db = db.client["nmdc_scratch"]
db.client.drop_database("nmdc_scratch")

In [ ]:
target_collection = {
    "Study": "study_set",
    "OmicsProcessing": "omics_processing_set",
    "Biosample": "biosample_set",
    "DataObject": "data_object_set",
    "MetagenomeAssembly": "metagenome_assembly_set",
    "MetaProteomicAnalysis": "metaproteomics_analysis_activity_set",
    "MetagenomeAnnotation": "metagenome_annotation_activity_set",
    "ReadQCAnalysisActivity": "read_QC_analysis_activity_set",
}
for collection in nmdc_database:
    docs = nmdc_database[collection]
    object_types = {d.get("type", "nmdc:")[5:] for d in docs} - {""}
    if any(d for d in docs if "type" not in d):
        print("some",collection,"docs have no type")
    print(collection, object_types)

In [ ]:
def reset_database(db):
    for coll_name in db.list_collection_names():
        db.drop_collection(coll_name)
        db[coll_name].create_index("id", unique=True)
        
reset_database(db)

In [ ]:
from collections import defaultdict

for source_collection in nmdc_database:
    docs = nmdc_database[source_collection]
    docs_per_target = defaultdict(list)
    for d in docs:
        type_ = d.get("type", "nmdc:")[5:]
        d_new = dissoc(d, "type")
        if "lat_lon" in d_new:
            d_new["lat_lon"].pop("type", None)
        for k_float in (
            "asm_score", "ctg_logsum", "ctg_powsum", "gap_pct", "gc_avg", "gc_std",
            "scaf_logsum", "scaf_powsum"):
            if k_float in d_new:
                d_new[k_float] = float(d_new[k_float]) 
        keys_with_term_ids = [
            k for k in d_new
            if isinstance(d_new[k], dict)
            and "term" in d_new[k]
            and "id" in d_new[k]["term"]
        ]
        for k in keys_with_term_ids:
            d_new = assoc_in(d_new, [k, "term"], d_new[k]["term"]["id"])
        
        key = target_collection[type_] if type_ else source_collection
        docs_per_target[key].append(d_new)
                
    for collection_name, docs in docs_per_target.items():
        print(collection_name)
        add_to_db(docs, db, collection_name=collection_name)

In [ ]:
import csv
import re

GOLD_ID_IDX = 5
IGSN_IDX = 2

igsn_golds = defaultdict(list)

gold_id_pattern = re.compile(r"Gb\d+")

with open('../src/data/FICUS_Soil_Gs0135149_Brodie-12-23-2020_PS.xlsx - Brodie_Gs0135149_Soil_Metadata.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        gold_id = row[GOLD_ID_IDX]
        igsn = row[IGSN_IDX]
        if gold_id_pattern.fullmatch(gold_id):
            igsn_golds[igsn].append(gold_id)

In [ ]:
from datetime import datetime

dt_pattern = re.compile(r"\d{2}-(?P<month>\w+)-\d{2} \d{2}\.\d{2}\.\d{2}\.(?P<ns>\d+) [A|P]M")
dt_format = "%d-%b-%y %I.%M.%S.%f %p"

def order_timestamps(timestamps):
    if not all(isinstance(ts, str) for ts in timestamps):
        raise Exception(f"{timestamps} not strings")
    as_datetimes = []
    for ts in timestamps:
        match = dt_pattern.search(ts)
        first, month, rest = ts.partition(match.group("month"))
        ts_new = first + month[0] + month[1:].lower() + rest
        ts_new = ts_new.replace(match.group("ns"), match.group("ns")[:-3]) # truncate to microseconds
        as_datetimes.append(datetime.strptime(ts_new, dt_format))
    sorted_dts = sorted(as_datetimes)
    return [dt.strftime(dt_format) for dt in sorted_dts]

In [ ]:
db.biosample_set.find_one()

In [ ]:
from pprint import pprint
from toolz import compose

er_xna_pattern = re.compile(r"(.*?)[-\s]+ER_[D|R]NA_(\d+)$")

def rstrip_name_ER_ID_include_number(d):
    s = get_in(["name"], d)
    m = er_xna_pattern.match(s)
    s_new = f"{m.group(1)} - {m.group(2)}" if m else s
    return assoc_in(d, ["name"], s_new)

def capitalize_location(d):
    s = get_in(["location"], d)
    if s is not None:
        s_new = (s[0].upper() + s[1:])
        return assoc_in(d, ["location"], s_new)
    else:
        return d

pipeline = compose(
    capitalize_location,
    rstrip_name_ER_ID_include_number,
    lambda d: dissoc(d, "_id", "id", "add_date", "mod_date", "identifier"),
)

In [ ]:
name = 'Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ER_DNA_115'

In [ ]:
m = er_xna_pattern.match(name)
m.group(1)

In [ ]:
m.group(2)

In [ ]:
!pip install dictdiffer

In [ ]:
from toolz import keyfilter

def pick(whitelist, d):
    return keyfilter(lambda k: k in whitelist, d)

In [ ]:
from pprint import pprint

from dictdiffer import diff
from toolz import get_in

merged_biosample_docs = []

for igsn, golds in igsn_golds.items():
    igsn_curie = "igsn:"+igsn
    to_change = list(db.biosample_set.find({"id": {"$in": [f"gold:{g}" for g in golds]}}))
    # No merge needed, just change of id.
    if len(to_change) == 1:
        merged = assoc_in(to_change[0], ["id"], igsn_curie)
        #merged = assoc_in(merged, ["identifier"], igsn_curie)
        merged_biosample_docs.append(merged)
        continue
    elif len(to_change) == 0:
        continue

    # Ensure that unification pipeline is adequate to resolve differences.
    distilled = list(map(pipeline, to_change))
    result = list(diff(distilled[0], distilled[1]))
    assert result == []
    
    # Produce a merged document
    earlier_ts, _ = order_timestamps([get_in(["add_date"], d) for d in to_change])
    merged = assoc_in(distilled[0], ["add_date"], earlier_ts)
    _, later_ts = order_timestamps([get_in(["mod_date"], d) for d in to_change])
    merged = assoc_in(merged, ["mod_date"], later_ts)
    merged = assoc_in(merged, ["id"], igsn_curie)
    merged = assoc_in(merged, ["identifier"], igsn_curie)
    
    merged_biosample_docs.append(merged)
    merged = None # defense against accidental reuse during next iteration.

assert len(merged_biosample_docs) == len(igsn_golds)

In [ ]:
from pymongo import DeleteMany, InsertOne
from toolz import concat

requests = [DeleteMany({"id": {"$in": ["gold:"+g for g in concat(igsn_golds.values())]}})]
requests.extend([InsertOne(d) for d in merged_biosample_docs])
result = db.biosample_set.bulk_write(requests)
result.deleted_count, result.inserted_count

In [ ]:
goldid_igsn = {}
for igsn, gids in igsn_golds.items():
    for gid in gids:
        goldid_igsn[gid] = igsn

In [ ]:
requests = []
to_replace = {"gold:"+k: "igsn:"+v for k, v in goldid_igsn.items()}

for doc in db.omics_processing_set.find({"has_input": {"$in": list(to_replace)}}):
    operations = {"$set": {
        "has_input": [to_replace.get(i, i) for i in doc["has_input"]],
    }}
    requests.append({"filter": {"_id": doc["_id"]}, "update": operations})

In [ ]:
from pymongo import UpdateOne

rv = db.omics_processing_set.bulk_write([UpdateOne(**r) for r in requests])

In [ ]:
rv.modified_count

In [ ]:
EMSL_IDS_IDX = 7
IGSN_IDX = 2

igsn_emsls = {}

emsl_ids_pattern = re.compile(r"\d+")

with open('../src/data/FICUS_Soil_Gs0135149_Brodie-12-23-2020_PS.xlsx - Brodie_Gs0135149_Soil_Metadata.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        emsl_ids = row[EMSL_IDS_IDX]
        igsn = row[IGSN_IDX]
        ids = emsl_ids_pattern.findall(emsl_ids)
        # XXX some rows have emsl ids but no IGSN, so igsn.strip() check here
        if igsn.strip() and ids:
            igsn_emsls[igsn] = ids

In [ ]:
emslid_igsn = {}
for igsn, eids in igsn_emsls.items():
    for eid in eids:
        emslid_igsn[eid] = igsn

In [ ]:
n_with_emsl_id = db.omics_processing_set.count_documents(
    {"id": {"$in": ["emsl:"+i for i in emslid_igsn]}})

In [ ]:
requests = []
to_replace = {"emsl:"+k: "igsn:"+v for k, v in emslid_igsn.items()}
to_replace.update({"emsl:output_"+k: "igsn:"+v for k, v in emslid_igsn.items()})

def omit(blacklist, d):
    return keyfilter(lambda k: k not in blacklist, d)

def sans_mongo_id(d):
    return omit(["_id"], d)


for doc in db.omics_processing_set.find({"has_input": {"$in": list(to_replace)}}):
    operations = {"$set": {
        "has_input": [to_replace.get(i, i) for i in doc["has_input"]],
    }}
    requests.append({"filter": {"_id": doc["_id"]}, "update": operations})

In [ ]:
if requests:
    rv = db.omics_processing_set.bulk_write([UpdateOne(**r) for r in requests])
    print(rv.modified_count)

In [ ]:
rows = []
with open("../src/data/2021-02-03-Stegen_biosample_linking_update - Sheet1.csv") as f:
    next(f) # skip header row
    for row in f:
        line = row.strip()
        tokens = line.split(",")
        if tokens[-1] == '':
            rows.append(tokens[:-1])
        else:
            rows.append(tokens)

In [ ]:
import json
from pprint import pprint


with open("../src/data/2021-02-04-stegen_biosample_template.json") as f:
    s = f.read()

try:
    stegen_sample_template = json.loads(s)
except json.JSONDecodeError:
    stegen_sample_template = json.loads(
        s.replace('\n', '')\
        .replace("$BIOSAMPLE_ID", '"$BIOSAMPLE_ID"')\
        .replace("“", '"')
    )

In [ ]:
import re

gold_pattern = re.compile(r"Gb\d+")

def prefix_sample_id(s):
    if ":" in s:
        return s
    elif re.fullmatch(gold_pattern, s):
        return "gold:" + s
    else:
        return "emsl:" + s

In [ ]:
omics = []
for i, row in enumerate(rows):
    omics.append({
        "omics_id": row[0],
        "omics_type": row[1],
        "sample_name": row[2],
        "sample_id": prefix_sample_id(row[3]),
        "new": len(row) > 4 and row[4] == "TRUE"
    })

In [ ]:
existing_ids = [
    d["id"] for d in
    db.biosample_set.find({"id": {"$in": [o["sample_id"] for o in omics]}}, ["id"])
]

In [ ]:
from toolz import assoc_in, get_in

def transform_in(doc, keys, fn):
    initial = get_in(keys, doc)
    transformed = fn(initial)
    return assoc_in(doc, keys, transformed)

In [ ]:
def fill_template(template, sample_id, sample_name):
    doc = assoc_in(template, ["id"], sample_id)
    doc = transform_in(
        doc, ["identifier", "has_raw_value"],
        lambda s: s.replace("$BIOSAMPLE_NAME", sample_name)
    )
    doc = transform_in(
        doc, ["name"],
        lambda s: s.replace("$BIOSAMPLE_NAME", sample_name)
    )
    return doc

In [ ]:
def term_subdocs_to_id_strings(doc):
    keys_with_term_ids = [
        k for k in doc
        if isinstance(doc[k], dict)
        and "term" in doc[k]
        and "id" in doc[k]["term"]
    ]
    for k in keys_with_term_ids:
        doc = assoc_in(doc, [k, "term"], doc[k]["term"]["id"])
    return doc

In [ ]:
new_samples = {}
for o in omics:
    if o["new"]:
        new_samples[o["sample_id"]] = o["sample_name"]

docs = []

for sample_id, sample_name in new_samples.items():
    doc = fill_template(stegen_sample_template, sample_id, sample_name)
    doc = term_subdocs_to_id_strings(doc)
    docs.append(doc)

In [ ]:
from toolz import get_in, assoc_in

def un_raw_value(doc, key):
    value = get_in([key, "has_raw_value"], doc)
    if value is not None:
        return assoc_in(doc, [key], value)
    else:
        return doc

raws = [
    "ecosystem",
    "collection_date",
    "community",
    "ecosystem_category",
    "ecosystem_subtype",
    "ecosystem_type",
    "geo_loc_name",
    "habitat",
    "identifier",
    "location",
    "ncbi_taxonomy_name",
    "sample_collection_site",
    "specific_ecosystem",
]
for key in raws:
    docs = [un_raw_value(d, key) for d in docs]

In [ ]:
docs

In [ ]:
from pymongo import ReplaceOne

rv = db.biosample_set.bulk_write([ReplaceOne({"id": doc["id"]}, doc, upsert=True) for doc in docs])

In [ ]:
rv.upserted_count

In [ ]:
omics = [
    transform_in(o, ["omics_id"], lambda s: "emsl:"+s if ":" not in s else s)
    for o in omics
]

In [ ]:
omics_ids = [o["omics_id"] for o in omics]

found_omics_ids = [
    d["id"] for d in
    db.omics_processing_set.find({"id": {"$in": omics_ids}},["id"])
]

In [ ]:
assert set(omics_ids) == set(found_omics_ids)

In [ ]:
omics_updates = {}
for o in omics:
    omics_updates[o["omics_id"]] = o

In [ ]:
from toolz import dissoc

replacing_omics_type = {}

requests = []

for doc in db.omics_processing_set.find({"id": {"$in": omics_ids}}):
    omics_type = get_in(["omics_type"], doc)
    updates = omics_updates[doc["id"]]
    if omics_type != updates["omics_type"]:
        replacing_omics_type[doc["id"]] = {"from": omics_type, "to": updates["omics_type"]}
    doc = assoc_in(doc, ["omics_type"], updates["omics_type"])
    doc = assoc_in(doc, ["has_input"], [updates["sample_id"]])
    requests.append(ReplaceOne({"id": doc["id"]}, dissoc(doc, "_id")))

In [ ]:
replacing_omics_type

In [ ]:
len(requests)

In [ ]:
rv = db.omics_processing_set.bulk_write(requests)

In [ ]:
rv.modified_count

In [ ]:
rows = []
with open("../src/data/2021-02-04-Brodie_biosample_linking_update - Sheet1.csv") as f:
    next(f) # skip header row
    for row in f:
        line = row.strip()
        tokens = line.split(",")
        if tokens[-1] == '':
            rows.append(tokens[:-1])
        else:
            rows.append(tokens)

In [ ]:
omics = []
for i, row in enumerate(rows):
    omics.append({
        "omics_id": "emsl:" + row[0].strip(),
        "omics_type": row[1].strip(),
        "sample_name": row[2].strip(),
        "sample_id": "igsn:" + row[3].strip(),
    })

In [ ]:
omics_ids = [o["omics_id"] for o in omics]

found_omics_ids = [
    d["id"] for d in
    db.omics_processing_set.find({"id": {"$in": omics_ids}},["id"])
]

In [ ]:
assert set(omics_ids) == set(found_omics_ids)

In [ ]:
omics_updates = {}
for o in omics:
    omics_updates[o["omics_id"]] = o

In [ ]:
replacing_omics_type = {}

requests = []

for doc in db.omics_processing_set.find({"id": {"$in": omics_ids}}):
    omics_type = get_in(["omics_type"], doc)
    updates = omics_updates[doc["id"]]
    if omics_type != updates["omics_type"]:
        replacing_omics_type[doc["id"]] = {"from": omics_type, "to": updates["omics_type"]}
    doc = assoc_in(doc, ["omics_type"], updates["omics_type"])
    doc = assoc_in(doc, ["has_input"], [updates["sample_id"]])
    requests.append(ReplaceOne({"id": doc["id"]}, dissoc(doc, "_id")))

In [ ]:
replacing_omics_type

In [ ]:
len(requests)

In [ ]:
rv = db.omics_processing_set.bulk_write(requests)

In [ ]:
rv.modified_count

Remove the 40 Wrighton EMSL omics_processing and data object documents that relate to isolates

In [ ]:
mfilter = {
    "part_of": ["gold:Gs0114675"],
    "processing_institution": "Environmental Molecular Sciences Lab"
}

db.omics_processing_set.count_documents(filter=mfilter)

In [ ]:
from pymongo import DeleteMany
from toolz import concat

requests = []


docs = list(db.omics_processing_set.find(mfilter, ["id", "has_output"]))
omics_processing_ids = [d["id"] for d in docs]
data_object_ids = list(concat(d["has_output"] for d in docs))

assert len(omics_processing_ids) == db.data_object_set.count_documents({"id": {"$in": data_object_ids}})

In [ ]:
rv1 = db.omics_processing_set.delete_many({"id": {"$in": omics_processing_ids}})
rv2 = db.data_object_set.delete_many({"id": {"$in": data_object_ids}})

In [ ]:
rv1.deleted_count, rv2.deleted_count

add 5 Brodie samples used at EMSL but not JGI

In [ ]:
biosamples = [
   {
      "name":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ",
      "description":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States",
      "lat_lon":{
         "has_raw_value":"38.9206 -106.9489",
         "latitude":38.9206,
         "longitude":-106.9489
      },
      "geo_loc_name":"USA: Colorado",
      "collection_date":"2017-05-09",
      "env_broad_scale":{
         "has_raw_value":"ENVO_00000446",
         "type":"ControlledTermValue"
      },
      "env_local_scale":{
         "has_raw_value":"ENVO_00000292",
         "type":"ControlledTermValue"
      },
      "env_medium":{
         "has_raw_value":"ENVO_00001998",
         "type":"ControlledTermValue"
      },
      "ecosystem":"Environmental",
      "ecosystem_category":"Terrestrial",
      "ecosystem_type":"Soil",
      "ecosystem_subtype":"Unclassified",
      "specific_ecosystem":"Unclassified",
      "depth":15,
      "ncbi_taxonomy_name":"soil metagenome",
      "community":"microbial communities",
      "location":"The East River watershed near Crested Butte, Colorado, USA",
      "habitat":"soil",
      "sample_collection_site":"soil",
      "add_date":"22-Jun-18 04.28.47.015000 PM",
      "mod_date":"01-Oct-19 09.41.01.459000 AM",
      "id":"igsn:IEWFS000I",
      "identifier":"igsn:IEWFS000I"
   },
   {
      "name":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ",
      "description":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States",
      "lat_lon":{
         "has_raw_value":"38.9206 -106.9489",
         "latitude":38.9206,
         "longitude":-106.9489
      },
      "geo_loc_name":"USA: Colorado",
      "collection_date":"2017-05-09",
      "env_broad_scale":{
         "has_raw_value":"ENVO_00000446",
         "type":"ControlledTermValue"
      },
      "env_local_scale":{
         "has_raw_value":"ENVO_00000292",
         "type":"ControlledTermValue"
      },
      "env_medium":{
         "has_raw_value":"ENVO_00001998",
         "type":"ControlledTermValue"
      },
      "ecosystem":"Environmental",
      "ecosystem_category":"Terrestrial",
      "ecosystem_type":"Soil",
      "ecosystem_subtype":"Unclassified",
      "specific_ecosystem":"Unclassified",
      "depth":15,
      "ncbi_taxonomy_name":"soil metagenome",
      "community":"microbial communities",
      "location":"The East River watershed near Crested Butte, Colorado, USA",
      "habitat":"soil",
      "sample_collection_site":"soil",
      "add_date":"22-Jun-18 04.28.47.015000 PM",
      "mod_date":"01-Oct-19 09.41.01.459000 AM",
      "id":"igsn:IEWFS000K",
      "identifier":"igsn:IEWFS000K"
   },
   {
      "name":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ",
      "description":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States",
      "lat_lon":{
         "has_raw_value":"38.9206 -106.9489",
         "latitude":38.9206,
         "longitude":-106.9489
      },
      "geo_loc_name":"USA: Colorado",
      "collection_date":"2017-05-09",
      "env_broad_scale":{
         "has_raw_value":"ENVO_00000446",
         "type":"ControlledTermValue"
      },
      "env_local_scale":{
         "has_raw_value":"ENVO_00000292",
         "type":"ControlledTermValue"
      },
      "env_medium":{
         "has_raw_value":"ENVO_00001998",
         "type":"ControlledTermValue"
      },
      "ecosystem":"Environmental",
      "ecosystem_category":"Terrestrial",
      "ecosystem_type":"Soil",
      "ecosystem_subtype":"Unclassified",
      "specific_ecosystem":"Unclassified",
      "depth":15,
      "ncbi_taxonomy_name":"soil metagenome",
      "community":"microbial communities",
      "location":"The East River watershed near Crested Butte, Colorado, USA",
      "habitat":"soil",
      "sample_collection_site":"soil",
      "add_date":"22-Jun-18 04.28.47.015000 PM",
      "mod_date":"01-Oct-19 09.41.01.459000 AM",
      "id":"igsn:IEWFS000B",
      "identifier":"igsn:IEWFS000B"
   },
   {
      "name":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ",
      "description":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States",
      "lat_lon":{
         "has_raw_value":"38.9206 -106.9489",
         "latitude":38.9206,
         "longitude":-106.9489
      },
      "geo_loc_name":"USA: Colorado",
      "collection_date":"2017-05-09",
      "env_broad_scale":{
         "has_raw_value":"ENVO_00000446",
         "type":"ControlledTermValue"
      },
      "env_local_scale":{
         "has_raw_value":"ENVO_00000292",
         "type":"ControlledTermValue"
      },
      "env_medium":{
         "has_raw_value":"ENVO_00001998",
         "type":"ControlledTermValue"
      },
      "ecosystem":"Environmental",
      "ecosystem_category":"Terrestrial",
      "ecosystem_type":"Soil",
      "ecosystem_subtype":"Unclassified",
      "specific_ecosystem":"Unclassified",
      "depth":15,
      "ncbi_taxonomy_name":"soil metagenome",
      "community":"microbial communities",
      "location":"The East River watershed near Crested Butte, Colorado, USA",
      "habitat":"soil",
      "sample_collection_site":"soil",
      "add_date":"22-Jun-18 04.28.47.015000 PM",
      "mod_date":"01-Oct-19 09.41.01.459000 AM",
      "id":"igsn:IEWFS000A",
      "identifier":"igsn:IEWFS000A"
   },
   {
      "name":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States - ",
      "description":"Soil microbial communities from the East River watershed near Crested Butte, Colorado, United States",
      "lat_lon":{
         "has_raw_value":"38.9206 -106.9489",
         "latitude":38.9206,
         "longitude":-106.9489
      },
      "geo_loc_name":"USA: Colorado",
      "collection_date":"2017-05-09",
      "env_broad_scale":{
         "has_raw_value":"ENVO_00000446",
         "type":"ControlledTermValue"
      },
      "env_local_scale":{
         "has_raw_value":"ENVO_00000292",
         "type":"ControlledTermValue"
      },
      "env_medium":{
         "has_raw_value":"ENVO_00001998",
         "type":"ControlledTermValue"
      },
      "ecosystem":"Environmental",
      "ecosystem_category":"Terrestrial",
      "ecosystem_type":"Soil",
      "ecosystem_subtype":"Unclassified",
      "specific_ecosystem":"Unclassified",
      "depth":15,
      "ncbi_taxonomy_name":"soil metagenome",
      "community":"microbial communities",
      "location":"The East River watershed near Crested Butte, Colorado, USA",
      "habitat":"soil",
      "sample_collection_site":"soil",
      "add_date":"22-Jun-18 04.28.47.015000 PM",
      "mod_date":"01-Oct-19 09.41.01.459000 AM",
      "id":"igsn:IEWFS000J",
      "identifier":"igsn:IEWFS000J"
   }
]

In [ ]:
add_to_db(biosamples, db, collection_name="biosample_set")

In [ ]:
assert db.biosample_set.count_documents({"id": {"$in": [d["id"] for d in biosamples]}}) == len(biosamples)